# Applied Probability and Randomized Algorithms: Coin flip simulations

Alviona Mancho | p3200098@aueb.gr

## Preliminaries

In [1]:
import random
from collections import Counter
import doctest

In [2]:
def print_message(prob_str, expected_val, actual_val):
    print("Expected {0}={1:.2f} \t Actual {0}={2:.2f}".format(prob_str, expected_val, actual_val))

## Task 1. A single fair coin: $Pr(Head) = 0.5$

### Definitions

- Bernoulli distribution is the discrete probability distribution of a random variable which takes the value $1$ with probability $p$ and $0$ with probability $1-p$. 
- We can denote value 1 as 'success' and 0 as 'failure'.
- To simulate a random variable $X$ ~ $Bern(p)$ we can generate a random number in the interval $[0,1)$; if the number turns out to be $<p$ then set the outcome $1$, else set it $0$. 

In [3]:
def bernoulli_sample(p, n):
    """Returns a sample where each instance ~ Bernoulli(p). The sample is constructed as a list of size n"""

    sample = [1 if random.random() < p else 0 for i in range(n)]
    return sample

Given a sample:
- $Pr(Success)  = Pr(X=1) = \frac{\#successes}{|sample|}$
- $Pr(Failure) = Pr(X=0) = \frac{\#failures}{|sample|} = 1 - Pr(success)$

In [4]:
def prob_success_failure(sample):
    """Returns the probability of success (1) and failure (0) in a Bernoulli sample: P(i=1), P(i=0)

    >>> sample = [1,0,1,0,0,0,1,1,0,1]
    >>> prob_success_failure(sample)
    (0.5, 0.5)

    >>> sample = [0,0,0,1,0,1,1,1,1,1]
    >>> prob_success_failure(sample)
    (0.6, 0.4)
    """

    size = len(sample)
    successes = sum(sample)
    failures = size - successes
    return successes/size, failures/size

Given a sample (a sequence of outcomes):
- $Pr(Success | Previously \; Success) = \frac{Pr (Success \,\cap\, Previously \,Success)}{Pr (Previously\, Success)} = \frac{\# success \,\&\, previously \,success}{\# previously \,success}$
- Likewise for $Pr(Success | Previously \,Failure)$

In [5]:
def cond_prob_success(sample):
    """Returns the conditional probability of success given the class of the previous instance (success or failure) in a Bernoulli sample: 
    P(current=1 | previous=1), P(current=1 | previous=0)
    
    >>> sample = [1,0,1,0,0,0,1,1,0,1]
    >>> cond_prob_success(sample)
    (0.25, 0.6)

    >>> sample = [0,0,0,1,0,1,1,1,1,1]
    >>> cond_prob_success(sample)
    (0.8, 0.5)
    """
    
    x = sum(map(lambda i: i[0]==1 and i[1]==1, zip(sample[1:], sample)))                #num of outcomes where current=1, previous=1
    y = sum(map(lambda i: i==1, sample[:-1]))                                           #num of outcomes where previous=1
    cond_prob_succ_prev_succ = x/y                                                      #P(current=1 | previous=1)

    x = sum(map(lambda i: i[0]==1 and i[1]==0, zip(sample[1:], sample)))                #num of outcomes where current=1, previous=0
    y = sum(map(lambda i: i==0, sample[:-1]))                                           #num of outcomes where previous=0
    cond_prob_succ_prev_fail = x/y                                                      #P(current=1 | previous=0)

    return cond_prob_succ_prev_succ, cond_prob_succ_prev_fail

Given a sample (a sequence of outcomes) of length $2n$, we can consider the sequence as a sequence of $n$ pairs e.g. $(1,1)$ for $(Success, Success)$.
- We can, thus, calculate the probability of each possible pair of outcomes: $(1,1)$, $(1,0)$, $(0,1)$, $(0,0)$

In [6]:
def prob_pairs(sample):
    """Returns the probability of each possible pair of success (1) and failure (0) in a Bernoulli sample: P(i=1, j=1), P(i=1, j=0), P(i=0, j=1), P(i=0, j=0)
    
    >>> sample = [1,0,1,0,0,0,1,1,0,1]
    >>> prob_pairs(sample)
    (0.2, 0.4, 0.2, 0.2)

    >>> sample = [0,0,0,1,0,1,1,1,1,1]
    >>> prob_pairs(sample)
    (0.4, 0.0, 0.4, 0.2)
    """

    size = len(sample)/2
    counts = Counter(zip(sample[0::2], sample[1::2]))   #view the sample as pairs of outcomes (using zip) and calculate the frequency of each possible pair
    
    succ_succ = counts[(1,1)]                           #num of (success, success)-pair outcomes                 
    succ_fail = counts[(1,0)]                           #num of (success, failure)-pair outcomes
    fail_succ = counts[(0,1)]                           #num of (failure, success)-pair outcomes
    fail_fail = counts[(0,0)]                           #num of (failure, failure)-pair outcomes
    
    prob_succ_succ = succ_succ/size
    prob_succ_fail = succ_fail/size
    prob_fail_succ = fail_succ/size
    prob_fail_fail = fail_fail/size

    return prob_succ_succ, prob_succ_fail, prob_fail_succ, prob_fail_fail

- We can, also, calculate conditional probabilities as before:

In [7]:
def cond_prob_pairs(sample):
    """Returns the conditional probability of each possible pair of outcomes consisting of instances of different classes (success (1) and failure (0)), given that their class is different: P(i=1, j=0 | i!=j), P(i=0, j=1 | i!=j)
    
    >>> sample = [1,0,1,0,0,1,1,1,0,1]
    >>> cond_prob_pairs(sample)
    (0.5, 0.5)

    >>> sample = [0,0,0,1,0,1,1,1,1,1]
    >>> cond_prob_pairs(sample)
    (0.0, 1.0)
    """
    
    x1 = sum(map(lambda i: i[0]==1 and i[1]==0, zip(sample[0::2], sample[1::2])))    #num of (success, failure)-pair outcomes
    x2 = sum(map(lambda i: i[0]==0 and i[1]==1, zip(sample[0::2], sample[1::2])))    #num of (failure, success)-pair outcomes
    y = x1+x2                                                                        #num of pairs consisting of instances of different classes = num (success, failure) + num (failure, success)
    cond_prob_succ_fail = x1/y                                                       #P(i=1, j=0 | i!=j)
    cond_prob_fail_succ = x2/y                                                       #P(i=0, j=1 | i!=j)
    return cond_prob_succ_fail, cond_prob_fail_succ

### Simulation

We can now simulate a (fair) coin flip as a random variable $X$ ~ $Bern(0.5)$ where $1$ denotes $Heads$ and $0$ denotes $Tails$. We are going to perform 1000 such flips and use this sample to calculate some actual probabilities (i.e. regarding the sample) and compare those to the respective theoretical probabilities.

- Note that the expected (theoretical) probabilities used for the comparisons are as follows:
    - $Pr(Head) = 0.5$, $Pr(Tail) = 0.5$

    - $Pr(Head | Previously \, Head) = 0.5$ due to assumption of independence. Same holds for $Pr(Head | Previously \, Tail)$

    - $Pr(Head, Head) = Pr(Head)*Pr(Head) = 0.5^{2} = 0.25$ etc. due to assumption of independence

    - $Pr(x \,Heads\, in\, a\, pair\, of\, 2\, flips) = {2 \choose x} * (0.5)^{x} * (0.5)^{2-x}$ for $x \in \{0,1,2\} $
    
    - $Pr(Head, Tail | Different \, instances) = \frac{Pr((Head, Tail) \,\cap\, Different \,instances)}{Pr(Different \,instances)} = \frac{Pr(Head, Tail)}{Pr(Different \,instances)} = \frac{0.5*0.5}{2*0.5*0.5} = 0.5$. 
    Same holds for $Pr(Tail, Head | Different \, instances)$

Below we also simulate an unfair coin flip as a random variable $X$ ~ $Bern(0.25)$. We repeat all the above calculations for this new (unfair) coin and make similar comparisons to the respective theoretical probabilities. Note that the expected (theoretical) probabilities used for the comparisons are calculated as shown above (but now $p=0.25$)

In [16]:
probs = [0.5, 0.25]
n = 1000

for p in probs:
    sample = bernoulli_sample(p, n)

    prob_head, prob_tail = prob_success_failure(sample)

    cond_prob_head_given_prev_head, cond_prob_head_given_prev_tail = cond_prob_success(sample)

    prob_head_head, prob_head_tail, prob_tail_head, prob_tail_tail = prob_pairs(sample)
    prob_one_head = prob_head_tail + prob_tail_head
    prob_two_heads = prob_head_head
    prob_zero_heads = 1 - (prob_one_head + prob_two_heads)

    cond_prob_head_tail, cond_prob_tail_head = cond_prob_pairs(sample)

    print()
    print("> Report: p={0}".format(p))

    print("A.")
    print_message("Pr(Head)", p, prob_head)
    print_message("Pr(Tail)", 1-p, prob_tail)

    print()

    print_message("Pr(Head|Prev Head)", p, cond_prob_head_given_prev_head)
    print_message("Pr(Head|Prev Tail)", p, cond_prob_head_given_prev_tail)

    print("\nB.")

    print_message("Pr(Head, Head)", p**2, prob_head_head)
    print_message("Pr(Head, Tail)", p*(1-p), prob_head_tail)
    print_message("Pr(Tail, Head)", (1-p)*p, prob_tail_head)
    print_message("Pr(Tail, Tail)", (1-p)**2, prob_tail_tail)

    print()

    print_message("Pr(0 Heads)", (1-p)**2, prob_zero_heads)
    print_message("Pr(1 Heads)", 2*p*(1-p), prob_one_head)
    print_message("Pr(2 Heads)", p**2, prob_two_heads)

    print("\nC.")

    print_message("Pr(Head, Tail|Different instances)", (p*(1-p)) / (2*p*(1-p)), cond_prob_head_tail)
    print_message("Pr(Tail, Head|Different instances)", (p*(1-p)) / (2*p*(1-p)), cond_prob_tail_head)


> Report: p=0.5
A.
Expected Pr(Head)=0.50 	 Actual Pr(Head)=0.48
Expected Pr(Tail)=0.50 	 Actual Pr(Tail)=0.52

Expected Pr(Head|Prev Head)=0.50 	 Actual Pr(Head|Prev Head)=0.50
Expected Pr(Head|Prev Tail)=0.50 	 Actual Pr(Head|Prev Tail)=0.46

B.
Expected Pr(Head, Head)=0.25 	 Actual Pr(Head, Head)=0.25
Expected Pr(Head, Tail)=0.25 	 Actual Pr(Head, Tail)=0.22
Expected Pr(Tail, Head)=0.25 	 Actual Pr(Tail, Head)=0.25
Expected Pr(Tail, Tail)=0.25 	 Actual Pr(Tail, Tail)=0.28

Expected Pr(0 Heads)=0.25 	 Actual Pr(0 Heads)=0.28
Expected Pr(1 Heads)=0.50 	 Actual Pr(1 Heads)=0.47
Expected Pr(2 Heads)=0.25 	 Actual Pr(2 Heads)=0.25

C.
Expected Pr(Head, Tail|Different instances)=0.50 	 Actual Pr(Head, Tail|Different instances)=0.48
Expected Pr(Tail, Head|Different instances)=0.50 	 Actual Pr(Tail, Head|Different instances)=0.52

> Report: p=0.25
A.
Expected Pr(Head)=0.25 	 Actual Pr(Head)=0.24
Expected Pr(Tail)=0.75 	 Actual Pr(Tail)=0.76

Expected Pr(Head|Prev Head)=0.25 	 Actual Pr(Hea

## Task 2. Two coins, one fair and one biased: $Pr_{0}(Head) = 0.5$,  $Pr_{1}(Head) = 0.25$

### Definitions

For this task we  need a different sample generator. Our purpose is to generate pairs of outcomes $(X, Y)$ where $X$~$Bern(p_i)$ and $Y$~Bern($p_i$) for $i\in\{0,1\}$. To do so, we first choose the $p_i$ randomly but fairly (i.e. both $p_0$ and $p_1$ are chosen with probability 0.5) and then use it 2 times to generate a pair of outcomes. We repeat this process $n$ times. In the end we are left with a sample (a sequence) of $2n$ outcomes ($n$ pairs put side by side). Also we keep track of which $p_i$ was used for each pair, using $0$ to represent $p_0$ and $1$ for $p_1$. 

In [9]:
def composite_bernoulli_sample(p0, p1, n):
    """Returns a sample of pairs of instances where each instance of the pair ~ Bernoulli(pi) and the probability used: 1 for p1 and 0 for p2. Before constructing a pair of instances, the probability pi for the pair is randomly selected among p1,p2
      in a fair manner. The sample is constructed as a list of size 2n. The probabilities are a list of size n"""
    
    sample = []
    probabilities = []
    for i in range(n):
        throw = random.random()
        prob = (throw < 0.5)*p0 + (throw >= 0.5)*p1

        sample.extend([1 if random.random() < prob else 0 for i in range(2)])
        probabilities.extend([1 if prob == p1 else 0])

    return sample, probabilities

Given a sample of length $2n$ we can calculate the posterior probability: 
- $Pr(Coin \, p_i \, was \, used | Outcome (0, 0) ) = \frac{Pr(Coin \, p_i \, was \, used \,\cap\, Outcome \, (0, 0) )}{Pr(Outcome \, (0, 0) )} = \frac{\# pairs \, where \, coin \, p_i \, was \, used}{\# (0,0) \, outcomes}$ etc.

In [10]:
def cond_prob_used_p_given_outcome(sample, probabilities, p_index, outcome):
    """Returns the posterior probability: P(p_index probability was used for the pair | the pair equals the outcome ).
    
    >>> sample = [1,0,1,0,0,0,1,1,0,1]
    >>> probabilities = [1, 0, 1, 1,0]
    >>> cond_prob_used_p_given_outcome(sample, probabilities, 1, (0,0))
    1.0

    >>> sample = [0,0,0,1,0,1,1,1,0,0]
    >>> probabilities = [1, 0, 1, 0, 0]
    >>> cond_prob_used_p_given_outcome(sample, probabilities, 0, (0,0))
    0.5
    """
    
    size = len(sample)/2

    x = sum(map(lambda i: i[0]==outcome[0] and i[1]==outcome[1] and i[2]==p_index, zip(sample[0::2], sample[1::2], probabilities)))/size  #prob of pairs where probability p_index was used for the pair & the pair equals the outcome
    
    p_succ_succ, p_succ_fail, p_fail_succ, p_fail_fail = prob_pairs(sample) 
    outcomes = {(1,1):p_succ_succ, (1,0):p_succ_fail, (0,1):p_fail_succ, (0,0): p_fail_fail}
    y = outcomes[outcome]                                                                                                                 #prob of outcome pair

    cond_prob_used_p_given_outcome = x/y                                                                                                  #P(p_index probability was used for the pair | the pair equals the outcome)
    return cond_prob_used_p_given_outcome

### Simulation

We can now simulate the selection of a coin between a fair coin (with $Pr(Head)=p_0$) and a biased coin (with $Pr(Tail)=p_1$) randomly but fairly (i.e. both coins $p_0$ and $p_1$ are chosen with probability 0.5) which we flip 2 times to generate a pair of outcomes. We are going to perform 1000 such coin selections and flips (i.e. 1000 pairs or 2000 outcomes put side by side) and use this sample to calculate some actual probabilities (i.e. regarding the sample) and compare those to the respective theoretical probabilities.
- Note that the expected (theoretical) probabilities used for the comparisons are as follows:
    - $Pr((Tail, Tail)) = Pr((Tail, Tail) | Fair \, coin) * Pr(Fair \, coin) + Pr((Tail, Tail) | Biased \, coin) * Pr(Biased \, coin) = 0.5^{2} * 0.5 + 0.75^{2} * 0.5 \approx 0.41$ due to the Law of Total Probability

    - $Pr(Biased \, coin| (Tail, Tail)) = \frac{Pr((Tail, Tail) | Biased \, coin)*Pr(Biased \, coin)}{Pr(Tail, Tail)} \approx \frac{0.75^{2} * 0.5}{0.41} \approx 0.69$ due to Bayes Theorem

In [11]:
p0 = 0.5
p1 = 0.25
n = 1000

sample, probabilities = composite_bernoulli_sample(p0, p1, n)
_, _, _, prob_tail_tail = prob_pairs(sample)
cond_prob_used_biased_given_tail_tail = cond_prob_used_p_given_outcome(sample, probabilities, 1, (0,0))

print("> Report:")

print_message("Pr(Tail, Tail)", (1-p0)**2*(0.5) + (1-p1)**2*(0.5), prob_tail_tail)
print_message("Pr(Biased coin|(Tail, Tail))", ((1-p1)**2 * 0.5)/((1-p0)**2*(0.5) + (1-p1)**2*(0.5)), cond_prob_used_biased_given_tail_tail)

> Report:
Expected Pr(Tail, Tail)=0.41 	 Actual Pr(Tail, Tail)=0.43
Expected Pr(Biased coin|(Tail, Tail))=0.69 	 Actual Pr(Biased coin|(Tail, Tail))=0.69
